# Informe del modelo de clasificación con redes neuronales

Este informe detalla el proceso y los resultados de un modelo de clasificación implementado utilizando redes neuronales con la biblioteca Keras. El objetivo del modelo es predecir las etiquetas de atributos a partir de características como la calificación (stars), la categoría (categoria) y el estado (state) de un conjunto de datos.

## 1. Preprocesamiento de los datos

En esta etapa, se realiza el preprocesamiento de los datos de entrada para prepararlos adecuadamente para el entrenamiento del modelo. Las siguientes acciones se llevan a cabo:

### 1.1. Selección de características

Se seleccionan las características relevantes para el modelo, que son la calificación (stars), la categoría (categoria) y el estado (state). Estas características se extraen del DataFrame original `df` y se almacenan en el DataFrame `X`.

### 1.2. Codificación de variables categóricas

Para poder utilizar las variables categóricas en el modelo, se realiza una codificación mediante la técnica de "one-hot encoding". Esto implica convertir las variables categóricas en variables numéricas binarias. En este caso, se aplica `pd.get_dummies(X)` para codificar las variables categóricas en columnas binarias correspondientes a cada categoría.

### 1.3. Codificación de etiquetas

Las etiquetas de atributos (y) se codifican utilizando el codificador de etiquetas (`LabelEncoder()`) de la biblioteca scikit-learn. Esto asigna un valor numérico a cada etiqueta de atributo, lo cual es necesario para el entrenamiento del modelo. Las etiquetas codificadas se almacenan en la variable `y_train_encoded`.

## 2. División del conjunto de datos

El conjunto de datos se divide en conjuntos de entrenamiento y prueba para evaluar el rendimiento del modelo. Se utiliza la función `train_test_split()` de scikit-learn para dividir los datos codificados (`X`) y las etiquetas codificadas (`y_train_encoded`) en conjuntos de entrenamiento (`X_train`, `y_train`) y prueba (`X_test`, `y_test`). En este caso, se asigna el 25% de los datos al conjunto de prueba y se utiliza una semilla aleatoria (`random_state`) para garantizar la reproducibilidad de los resultados.

## 3. Definición del modelo

El modelo se define como una secuencia de capas utilizando la API `Sequential` de Keras. El modelo consta de tres capas densas (totalmente conectadas) con funciones de activación ReLU, seguidas de una capa de salida con una función de activación softmax. La capa de entrada tiene una dimensión igual al número de características en los datos de entrenamiento (`X_train.shape[1]`), y la capa de salida tiene una dimensión igual al número máximo de etiquetas codificadas (`np.max(y_train) + 1`).

## 4. Compilación y entrenamiento del modelo

Antes de entrenar el modelo, se debe compilar especificando la función de pérdida (`loss`), el optimizador (`optimizer`) y las métricas que se utilizarán para evaluar el rendimiento del modelo (`metrics`). En este caso, se utiliza la función de pérdida `sparse_categorical_crossentropy` adecuada para problemas de clasificación con múltiples clases. El optimizador seleccionado es `adam`, que es un algoritmo de optimización popular en el aprendizaje profundo.

A continuación, se entrena el modelo utilizando el conjunto de entrenamiento (`X_train`, `y_train`) durante un número determinado de épocas (`epochs`) y un tamaño de lote (`batch_size`). Además, se utiliza una validación cruzada del 20% del conjunto de entrenamiento (`validation_split`) para monitorear el rendimiento del modelo durante el entrenamiento y evitar el sobreajuste.

## 5. Evaluación del modelo

Una vez finalizado el entrenamiento, se evalúa el rendimiento del modelo utilizando el conjunto de prueba (`X_test`, `y_test`). Se calcula la pérdida (`test_loss`) y la precisión (`test_acc`) del modelo en este conjunto de datos.

## 6. Predicciones

Se utilizan las características codificadas del conjunto de prueba (`X_test`) para realizar predicciones con el modelo entrenado. Las predicciones se almacenan en la variable `predictions` y representan las probabilidades de pertenencia a cada clase de atributo.

---

Este informe describe el proceso de construcción, entrenamiento y evaluación de un modelo de clasificación utilizando redes neuronales. El modelo utiliza las características de calificación, categoría y estado para predecir las etiquetas de atributos. Los resultados obtenidos se basan en las métricas de pérdida y precisión del modelo en el conjunto de prueba.


## Ahora si, el código... ##

In [80]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import re
import mtranslate as mt

Se importan librerias y se carga el dataframe listo para el modelo

In [17]:
df = pd.read_csv("dataset2.csv", low_memory=False)

Se crean variables y la red neuronal + el entrenamiento y testeo

In [18]:
X = df[['stars', 'categoria', 'state']]
y = df['atributos']

# filtered_indices = (y != 'AcceptsInsurance: None')
# X = X[filtered_indices]
# y = y[filtered_indices]

X = pd.get_dummies(X)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y)
y_train_encoded = y_train_encoded.astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y_train_encoded, test_size=0.25, random_state=42)

model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(np.max(y_train) + 1, activation='softmax') 
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=15, batch_size=100, validation_split=0.2)

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Precisión en el conjunto de prueba: {test_acc}')

predictions = model.predict(X_test)

Epoch 1/15
9110/9110 [==============================] - 41s 4ms/step - loss: 3.9026 - accuracy: 0.0846 - val_loss: 3.6592 - val_accuracy: 0.0877
Epoch 2/15
9110/9110 [==============================] - 40s 4ms/step - loss: 3.6652 - accuracy: 0.0876 - val_loss: 3.6612 - val_accuracy: 0.0878
Epoch 3/15
9110/9110 [==============================] - 44s 5ms/step - loss: 3.6605 - accuracy: 0.0875 - val_loss: 3.6927 - val_accuracy: 0.0877
Epoch 4/15
9110/9110 [==============================] - 43s 5ms/step - loss: 3.6576 - accuracy: 0.0875 - val_loss: 3.6587 - val_accuracy: 0.0875
Epoch 5/15
9110/9110 [==============================] - 44s 5ms/step - loss: 3.6550 - accuracy: 0.0874 - val_loss: 3.6533 - val_accuracy: 0.0875
Epoch 6/15
9110/9110 [==============================] - 40s 4ms/step - loss: 3.6524 - accuracy: 0.0874 - val_loss: 3.6487 - val_accuracy: 0.0877
Epoch 7/15
9110/9110 [==============================] - 37s 4ms/step - loss: 3.6493 - accuracy: 0.0873 - val_loss: 3.6572 - val_ac

Guardado del modelo

In [19]:
model.save('modelo_at_1')

INFO:tensorflow:Assets written to: modelo_at_1\assets


INFO:tensorflow:Assets written to: modelo_at_1\assets


**Se crean funciones para ejecutar consultas tanto con el dataset original, en inglés, como en español y con la cantidad deseada de atributos a visualizar**

In [67]:
def hacer_predicciones(modelo, datos_nuevos, top_n=5):
    datos_codificados = pd.get_dummies(datos_nuevos[['stars', 'categoria', 'state']])
    predicciones = modelo.predict(datos_codificados)
    top_clases = np.argsort(-predicciones)[:, :top_n]
    etiquetas_predichas = []
    
    for ejemplo in top_clases:
        atributos = label_encoder.inverse_transform(ejemplo)
        atributos_divididos = [re.sub(r'(?<!^)(?=[A-Z])', ' ', attr.split(': ')[0]) + ': ' + attr.split(': ')[1] for attr in atributos]
        atributos_traducidos = [mt.translate(attr, "es", "en") for attr in atributos_divididos]
        etiquetas_predichas.append(atributos_traducidos)
    
    return etiquetas_predichas

In [21]:
def hacer_predicciones2(modelo, datos_nuevos, top_n=5):
    datos_codificados = pd.get_dummies(datos_nuevos[['stars', 'categoria', 'state']])
    predicciones = modelo.predict(datos_codificados)
    top_clases = np.argsort(-predicciones)[:, :top_n]
    etiquetas_predichas = []
    
    for ejemplo in top_clases:
        etiquetas_predichas.append(label_encoder.inverse_transform(ejemplo))
    
    return list(etiquetas_predichas)

In [22]:
def hacer_predicciones3(modelo, datos_nuevos, top_n=5):
    datos_codificados = pd.get_dummies(datos_nuevos[['stars', 'categoria', 'state']])
    predicciones = modelo.predict(datos_codificados)
    top_clases = np.argsort(-predicciones)[:, :top_n]
    etiquetas_predichas = []
    
    for ejemplo in top_clases:
        atributos = label_encoder.inverse_transform(ejemplo)
        atributos_divididos = [attr.split(':')[0] for attr in atributos if ':' in attr]
        atributos_traducidos = [mt.translate(attr, "es", "en") for attr in atributos_divididos]
        etiquetas_predichas.append(atributos_traducidos)
    
    return etiquetas_predichas

In [23]:
def hacer_predicciones4(modelo, datos_nuevos, top_n=10):
    datos_codificados = pd.get_dummies(datos_nuevos[['stars', 'categoria', 'state']])
    predicciones = modelo.predict(datos_codificados)
    top_clases = np.argsort(-predicciones)[:, :top_n]
    etiquetas_predichas = []
    
    for ejemplo in top_clases:
        atributos = label_encoder.inverse_transform(ejemplo)
        atributos_divididos = [re.sub(r'(?<!^)(?=[A-Z])', ' ', attr.split(': ')[0]) + ': ' + attr.split(': ')[1] for attr in atributos]
        atributos_traducidos = [mt.translate(attr, "es", "en") for attr in atributos_divididos]
        etiquetas_predichas.append(atributos_traducidos)
    
    return etiquetas_predichas

In [26]:
def hacer_predicciones5(modelo, datos_nuevos, top_n=5):
    datos_codificados = pd.get_dummies(datos_nuevos[['stars', 'categoria', 'state']])
    predicciones = modelo.predict(datos_codificados)
    top_clases = np.argsort(-predicciones)[:, :top_n]
    etiquetas_predichas = []
    
    for ejemplo in top_clases:
        atributos = label_encoder.inverse_transform(ejemplo)
        atributos_divididos = []
        
        for attr in atributos:
            if ':' in attr and 'Verdadero' not in attr:
                attr_dividido = attr.split(':')[0]
                atributos_divididos.append(attr_dividido)
        
        atributos_traducidos = [mt.translate(attr, "es", "en") for attr in atributos_divididos]
        etiquetas_predichas.append(atributos_traducidos)
    
    return etiquetas_predichas


*El labelEconder de abajo sirve para ejecutar solo cuando el modelo ya esta cargado y no se quiere volver a entrenar para declarar la variable (ojo, sino las funciones pueden arrojar errores!)*

In [7]:
# label_encoder = LabelEncoder()
# label_encoder.fit(df['atributos'])

LabelEncoder()

## **Algunas consultas...** ##

In [24]:
modelo_cargado = keras.models.load_model('modelo_at_1')
nuevos_datos = pd.DataFrame({'stars': [2], 'categoria': ['Restaurante'], 'state': ['California']})
etiquetas_predichas = hacer_predicciones(modelo_cargado, nuevos_datos, top_n=5)
etiquetas_predichas

1/1 [==============================] - 0s 162ms/step


[['Rango de precios de restaurantes2: 3',
  'El negocio acepta tarjetas de crédito: verdadero',
  'Restaurantes Rango de precios2: 2',
  'Estacionamiento de bicicletas: verdadero',
  'Estacionamiento comercial en la calle: Verdadero']]

In [9]:
modelo_cargado = keras.models.load_model('modelo_at_1')
nuevos_datos = pd.DataFrame({'stars': [4], 'categoria': ['Restaurante'], 'state': ['California']})
etiquetas_predichas = hacer_predicciones3(modelo_cargado, nuevos_datos, top_n=5)
etiquetas_predichas

1/1 [==============================] - 0s 134ms/step


[['NegocioAceptaCréditoTarjetas',
  'Solo por cita',
  'Calle de estacionamiento de negocios',
  'BicicletaAparcamiento',
  'Silla de ruedas accesible']]

In [68]:
modelo_cargado = keras.models.load_model('modelo_at_1')
nuevos_datos = pd.DataFrame({'stars': [2], 'categoria': ['Restaurante'], 'state': ['California']})
etiquetas_predichas = hacer_predicciones(modelo_cargado, nuevos_datos, top_n=10)
etiquetas_predichas

1/1 [==============================] - 0s 67ms/step


[['Rango de precios de restaurantes2: 3',
  'El negocio acepta tarjetas de crédito: verdadero',
  'Restaurantes Rango de precios2: 2',
  'Estacionamiento de bicicletas: verdadero',
  'Estacionamiento comercial en la calle: Verdadero',
  'Estacionamiento comercial: verdadero',
  'Restaurantes Rango de precios2: 4',
  'Rango de precios de restaurantes2: 1',
  'Accesible para sillas de ruedas: Verdadero',
  'Estacionamiento comercial: garaje']]

*¡Hasta Aqui la red neuronal!*